In [86]:
import pandas as pd
import json


In [87]:

# Load the JSON data
file_path = "C:/Users/Sergi/Desktop/FlipWise/flashcards_user1_data.json"

with open(file_path, 'r') as f:
    data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Sergi/Desktop/FlipWise/flashcards_user1_data.json'

In [ ]:
# Convert to DataFrames for easier manipulation
users_df = pd.DataFrame(data['users'])
decks_df = pd.DataFrame(data['decks'])
cards_df = pd.DataFrame(data['cards'])
reviews_df = pd.DataFrame(data['reviews'])

In [ ]:
# Preview the structure
users_df.head(), decks_df.head(), cards_df.head(), reviews_df.head()

(   id     username                     email           password  \
 0   1  reesethomas  hayesjeffrey@hotmail.com  hashed_password_1   
 
             created_at           updated_at  
 0  2025-01-17 20:11:29  2025-07-22 12:16:39  ,
    id    name                                      description  user_id  \
 0   1  Deck 1                 Leader who article look husband.        1   
 1   2  Deck 2                  Should share face build market.        1   
 2   3  Deck 3        Behind question exist rich prevent trade.        1   
 3   4  Deck 4  Democratic change vote participant institution.        1   
 4   5  Deck 5                            Call door population.        1   
 
             created_at           updated_at  
 0  2025-02-11 22:16:37  2025-07-10 09:38:12  
 1  2025-06-17 05:59:44  2025-03-10 05:35:39  
 2  2025-02-28 09:34:51  2025-05-10 11:09:33  
 3  2025-04-22 21:18:04  2025-04-03 06:58:02  
 4  2025-05-21 15:29:17  2025-05-29 16:27:41  ,
    id                    

In [ ]:
# Crée une colonne 'correct' dans reviews_df pour coder True/False
reviews_df['is_correct'] = reviews_df['rating'] == 'correct'

# Pour chaque carte, déterminer s'il y a AU MOINS une réponse correcte
correct_cards = reviews_df.groupby('card_id')['is_correct'].any().reset_index()
correct_cards.columns = ['card_id', 'has_correct_answer']

correct_cards.head()



,card_id,has_correct_answer
0,1,True
1,2,True
2,3,False
3,4,True
4,5,True


In [ ]:
# Renommer la colonne 'id' en 'card_id' pour permettre la fusion
cards_df = cards_df.rename(columns={'id': 'card_id'})

# Fusionner avec les données de réponses correctes
cards_with_correct = cards_df.merge(correct_cards, on='card_id', how='left')

# Remplir les NaN avec False (les cartes sans réponse correcte)
cards_with_correct['has_correct_answer'] = cards_with_correct['has_correct_answer'].fillna(False)

# Calculer le progrès par deck
deck_progress = cards_with_correct.groupby('deck_id').agg(
    total_cards=('card_id', 'count'),
    correct_cards=('has_correct_answer', 'sum')
).reset_index()

deck_progress['progress_percent'] = (deck_progress['correct_cards'] / deck_progress['total_cards']) * 100

# Ajouter les noms des decks depuis decks_df
deck_progress = deck_progress.merge(decks_df[['id', 'name']], left_on='deck_id', right_on='id')
deck_progress = deck_progress[['name', 'total_cards', 'correct_cards', 'progress_percent']]

# Affichage final
deck_progress



,name,total_cards,correct_cards,progress_percent
0,Deck 1,6,4,66.666667
1,Deck 2,6,6,100.000000
2,Deck 3,6,4,66.666667
3,Deck 4,6,2,33.333333
4,Deck 5,6,2,33.333333


In [ ]:
import os
import json

# Créer le dossier 'static' s’il n'existe pas encore
os.makedirs('static', exist_ok=True)

# Convertir le DataFrame en liste de dictionnaires
deck_progress_json = deck_progress.to_dict(orient='records')

# Sauvegarder dans un fichier JSON
with open('static/deck_progress.json', 'w') as f:
    json.dump(deck_progress_json, f, indent=4)

print("✅ Fichier deck_progress.json généré avec succès dans le dossier 'static'")


✅ Fichier deck_progress.json généré avec succès dans le dossier 'static'


In [ ]:
import json

# Ce code suppose que tu as déjà deck_progress sous forme d'un DataFrame

deck_progress_json = deck_progress.to_dict(orient='records')

with open('static/deck_progress.json', 'w') as f:
    json.dump(deck_progress_json, f, indent=4)


In [ ]:
import json
import pandas as pd

# Charger les données du fichier JSON principal
with open('static/flashcards_user1_data.json') as f:
    data = json.load(f)

# Extraction des statuts
cards_df = pd.DataFrame(data["cards"])
status_counts = cards_df['status'].value_counts().to_dict()

# Sécuriser les valeurs (s'assurer que chaque statut existe)
acquises = status_counts.get("acquise", 0)
a_reviser = status_counts.get("à réviser", 0)
total = acquises + a_reviser

# Calcul des pourcentages
progress_data = {
    "acquises": acquises,
    "à réviser": a_reviser,
    "acquises_percent": round(100 * acquises / total, 2) if total > 0 else 0,
    "à_reviser_percent": round(100 * a_reviser / total, 2) if total > 0 else 0
}

# Sauvegarde dans un fichier JSON
with open('static/card_status_progress.json', 'w') as f:
    json.dump(progress_data, f, indent=4)
